# QGS model: Simple run example

First, setting the path and loading of some modules

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from numba import njit

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.cm as cm

Initializing the random number generator (for reproducibility). -- Disable if needed.

In [ ]:
np.random.seed(21217)

Importing the model's modules

In [ ]:
from params.params import QgParams
from integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
from model.definition import create_tendencies, create_linearized_tendencies

## Systems definition

General parameters

In [ ]:
# Time parameters
dt = 0.1
# Saving the model state n steps
write_steps = 5

number_of_trajectories = 1
number_of_perturbed_trajectories = 10


Creating the tendencies function

In [ ]:
model_parameters = QgParams()
model_parameters.set_max_modes(2, 2)
f = create_tendencies(model_parameters)
Df = create_linearized_tendencies(model_parameters)

## Time integration

Defining an integrator

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

Start on a random initial condition to obtain another initial condition on the attractors

In [ ]:
%%time
ic = np.random.rand(model_parameters.ndim)*0.1
integrator.integrate(0., 100000., dt, ic=ic, write_steps=0)
time, ic = integrator.get_trajectories()

Now integrate to obtain a trajectory on the attractor

In [ ]:
%%time
integrator.integrate(0., 10000., dt, ic=ic, write_steps=write_steps)
time, traj = integrator.get_trajectories()

Plotting the result in 3D and 2D

In [ ]:
fig = plt.figure(figsize=(10, 8))
axi = fig.gca(projection='3d')

axi.scatter(traj[0], traj[1], traj[2], s=0.2);

axi.set_xlabel('$'+model_parameters.latex_var_string[0]+'$')
axi.set_ylabel('$'+model_parameters.latex_var_string[1]+'$')
axi.set_zlabel('$'+model_parameters.latex_var_string[2]+'$');

In [ ]:
varx = 0
vary = 1
plt.figure(figsize=(10, 8))

plt.plot(traj[varx], traj[vary], marker='o', ms=0.1, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[0]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[1]+'$');

In [ ]:
var = 10
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*time, traj[var])

plt.xlabel('time (days)')
plt.ylabel('$'+model_parameters.latex_var_string[var]+'$');

## Initial condition sensitivity analysis example

In [ ]:
tgls_integrator = RungeKuttaTglsIntegrator()

In [ ]:
tgls_integrator.set_func(f, Df)

In [ ]:
ic = np.random.randn(4,20)

In [ ]:
tangent_ic = 0.001*np.random.randn(model_parameters.ndim)

In [ ]:
tangent_ic.shape

In [ ]:
%%time 

tgls_integrator.integrate(0., 100., dt=dt, write_steps=write_steps, ic=ic, tg_ic=tangent_ic)

In [ ]:
tgls_integrator.tg_ic.shape

In [ ]:
tt, traj, deltay = tgls_integrator.get_trajectories()